# Exercise 2.1

Where 

$|r\rangle = \frac{1}{\sqrt{2}}|u\rangle + \frac{1}{\sqrt{2}}|d\rangle$

and 

$|l\rangle = \frac{1}{\sqrt{2}}|u\rangle - \frac{1}{\sqrt{2}}|d\rangle$

Prove that

$|r\rangle \perp |l\rangle$

We know that if two vectors are orthogonal their inner product is 0.

$|r\rangle \perp |l\rangle \iff \langle r |l\rangle = 0$

$\langle r |l\rangle = \frac{1}{\sqrt{2}}\frac{1}{\sqrt{2}} - \frac{1}{\sqrt{2}}\frac{1}{\sqrt{2}} = 0$

$\square$

# Exercise 2.2
Where

$|i\rangle = \frac{1}{\sqrt{2}}|u\rangle + \frac{i}{\sqrt{2}}|d\rangle$

$|o\rangle = \frac{1}{\sqrt{2}}|u\rangle - \frac{i}{\sqrt{2}}|d\rangle$

$|r\rangle = \frac{1}{\sqrt{2}}|u\rangle + \frac{1}{\sqrt{2}}|d\rangle$

$|l\rangle = \frac{1}{\sqrt{2}}|u\rangle - \frac{1}{\sqrt{2}}|d\rangle$

Prove that $|i\rangle$ and $|o\rangle$ satisfy the following conditions:

$\langle i|o\rangle  = 0$

2.8

$\langle o|u \rangle \langle u|o\rangle = \frac{1}{2}$

$\langle o|d \rangle \langle d|o \rangle = \frac{1}{2}$

$\langle i|u \rangle \langle u|i \rangle= \frac{1}{2}$

$\langle i|d \rangle \langle d|i \rangle= \frac{1}{2}$

2.9

$\langle o|r \rangle \langle r|o\rangle = \frac{1}{2}$

$\langle o|l \rangle \langle l|o \rangle = \frac{1}{2}$

$\langle i|r \rangle \langle r|i \rangle= \frac{1}{2}$

$\langle i|l \rangle \langle l|i \rangle= \frac{1}{2}$


In [181]:
# The qutip library is useful for numerical calculations with quantum mechanics

from qutip import *
import numpy as np
import itertools

# Define the basis ket vectors
space = {'u' : basis(2, 0), 'd' : basis(2, 1)}

 # Define other kets as linear combinations of the basis kets
space['I'] = (space['u'] + 1j * space['d']).unit()
space['O'] = (space['u'] - 1j * space['d']).unit()
space['L'] = (space['u'] + space['d']).unit()
space['R'] = (space['u'] - space['d']).unit()

# Computing <i|o>
inner_product = space['I'].dag() * space['u']
print(f'<i|u> = {inner_product[0]}')

# computing equations 2.8 and 2.9
for pair in itertools.product(['I', 'O'], ['u', 'd', 'L', 'R']):
    p1 = space[pair[0]].dag() * space[pair[1]]
    p2 = space[pair[1]].dag() * space[pair[0]]
    print(f'<{pair[0]}|{pair[1]}><{pair[1]}|{pair[0]}> = {p1*p2[0][0]}')

<i|u> = [[0.70710678+0.j]]
<I|u><u|I> = [0.5+0.j]
<I|d><d|I> = [0.5+0.j]
<I|L><L|I> = [0.5+0.j]
<I|R><R|I> = [0.5+0.j]
<O|u><u|O> = [0.5+0.j]
<O|d><d|O> = [0.5+0.j]
<O|L><L|O> = [0.5+0.j]
<O|R><R|O> = [0.5+0.j]


# Exercise 2.3

In [178]:
'''sympy is more suitable for this symbolic computation'''
from sympy import *
from sympy.physics.quantum import Ket

# basis vectors
u = Ket('u')
d = Ket('d')

alpha, beta, gamma, delta = symbols('alpha beta gamma delta', real=True)

# Define the other kets as linear combinations of the basis kets
I = alpha*u + beta*d
O = gamma*u + delta*d

# Normalization condition for any linear combination of the basis kets
condition_1 = Eq(abs(alpha)**2 + abs(beta)**2, 1)
condition_2 = Eq(abs(gamma)**2 + abs(delta)**2, 1)

# Solution conditions
condition_alpha = Eq(conjugate(alpha)*alpha, 1/2)
condition_beta = Eq(conjugate(beta)*beta, 1/2)
condition_gamma = Eq(conjugate(gamma)*gamma, 1/2)
condition_delta = Eq(conjugate(delta)*delta, 1/2)

# use sympy to solve the equations
solution_alpha = solve((condition_1, condition_alpha), (alpha,beta))
solution_beta = solve((condition_1, condition_beta), (alpha,beta))
solution_gamma = solve((condition_2, condition_gamma), (gamma,delta))
solution_delta = solve((condition_2, condition_delta), (gamma,delta))

# check the solutions
assert round(solution_alpha[0][0]**2,1) == 0.5
assert round(solution_beta[0][0]**2,1) == 0.5
assert round(solution_gamma[0][0]**2,1) == 0.5
assert round(solution_delta[0][0]**2,1) == 0.5

In [215]:
class Ket:
    def __init__(self, values):
        if type(values) == Ket:
            values = values.values

        elif type(values) == Bra:
            values = [val.conjugate() for val in values.values]

        assert type(values) == list

        self.values = values

    def __str__ (self):
        vals = [f'|{val}>, ' for val in self.values]
        return ''.join(vals)

    def __mul__(self, other):
        return sum([a*b for a, b in zip(Bra(self).values, Ket(other).values)])

    def __rmul__(self, other):
        return sum([a*b for a, b in zip(Bra(other).values, self.values)])
    
    def __add__(self, other):
        return Ket([a+b for a, b in zip(self.values, Ket(other).values)])
    
    def __sub__(self, other):
        return Ket([a-b for a, b in zip(self.values, Ket(other).values)])
    
    def __eq__(self, other):
        return self.values == other.values and type(self) == type(other)
    
class Bra:
    def __init__(self, values):
        if type(values) == Bra:
            values = values.values
        elif type(values) == Ket:
            values = [val.conjugate() for val in values.values]
        assert type(values) == list

        self.values = values

    def __str__ (self):
        vals = [f'<{val}|, ' for val in self.values]
        return ''.join(vals)

    def __mul__(self, other):
        return sum([a*b for a, b in zip(self.values, Ket(other).values)])

    def __rmul__(self, other):
        return sum([a*b for a, b in zip(Bra(other).values, Ket(self).values)])
    
    def __add__(self, other):
        return Bra([a+b for a, b in zip(self.values, Bra(other).values)])
    
    def __sub__(self, other):
        return Bra([a-b for a, b in zip(self.values, Bra(other).values)])
    
    def __eq__(self, other):
        return self.values == other.values and type(self) == type(other)